In [58]:
from openai import  OpenAI
from dotenv import load_dotenv
import re
import pandas as pd

In [59]:
load_dotenv()

True

In [60]:
SRC_LANG = "french"
DST_LANG = "spanish"

In [61]:
def get_prompt(src_lang: str, dst_lang: str, input: list):

    input_str = ',\n'.join(f"{item}" for item in input)

    print(type(input_str))
    print(input)

    prompt = """
    Using the following list of words in {src_lang}, extract the corresponding words and translate it to {dst_lang},
    returning a json file object containing the original words as key and the translation as value.

    START EXAMPLE
    Input: {{
    [Vocabulaire,
    i ampoule n.f.,
    appareil n.m.,
    aspirateur n.m.,
    baignoire n.f.,
    optimiste adj.,
    augmenter,
    ]
    }}
    Output: 
    {{
        Le Vocabulaire: El Vocabulario,
        L'ampoule: La Bombilla,
        Le aspirateur: La aspiradora,
        La baignoire: La bañera,
        optimiste: optimista,
        augmenter: aumentar,
    }}
    END EXAMPLE
    Input:
    {input_str}

    Output: 
    """

    return prompt.format(src_lang=src_lang, dst_lang=dst_lang, input_str=input_str)



## Corregir el caso de verbos, adjetivos, adverbios
# Agregar mas ejemplos y mas variados

In [62]:
input = ['Vocabulaire',
 'i ampoule n.f.',
 'appareil n.m.',
 'aspirateur n.m.',
 'baignoire n.f.',
 'bouton n.m.',
 'bricolage n.m.',
 'bricoleur n.m.',
 'couvert n.m.',
 'égalité n.f.',
 'entretien n.m.',
 'évier n.m.',
 'four n.m.',
 'intérieur n.m.',
 'kit n.m.',
 'lavabo n.m.',
 'lave-linge n.m.',
 'lave-vaisselle n.m.',
 'lessive n.f.',
 'linge n.m.',
 'lit n.m.',
 'ménage n.m.',
 'meuble n.m.',
 'miroir n.m.',
 'poubelle n-f.',
]

prompt = get_prompt(SRC_LANG, DST_LANG, input)

<class 'str'>
['Vocabulaire', 'i ampoule n.f.', 'appareil n.m.', 'aspirateur n.m.', 'baignoire n.f.', 'bouton n.m.', 'bricolage n.m.', 'bricoleur n.m.', 'couvert n.m.', 'égalité n.f.', 'entretien n.m.', 'évier n.m.', 'four n.m.', 'intérieur n.m.', 'kit n.m.', 'lavabo n.m.', 'lave-linge n.m.', 'lave-vaisselle n.m.', 'lessive n.f.', 'linge n.m.', 'lit n.m.', 'ménage n.m.', 'meuble n.m.', 'miroir n.m.', 'poubelle n-f.']


In [63]:
print(prompt)


    Using the following list of words in french, extract the corresponding words and translate it to spanish,
    returning a json file object containing the original words as key and the translation as value.

    START EXAMPLE
    Input: {
    [Vocabulaire,
    i ampoule n.f.,
    appareil n.m.,
    aspirateur n.m.,
    baignoire n.f.,
    optimiste adj.,
    augmenter,
    ]
    }
    Output: 
    {
        Le Vocabulaire: El Vocabulario,
        L'ampoule: La Bombilla,
        Le aspirateur: La aspiradora,
        La baignoire: La bañera,
        optimiste: optimista,
        augmenter: aumentar,
    }
    END EXAMPLE
    Input:
    Vocabulaire,
i ampoule n.f.,
appareil n.m.,
aspirateur n.m.,
baignoire n.f.,
bouton n.m.,
bricolage n.m.,
bricoleur n.m.,
couvert n.m.,
égalité n.f.,
entretien n.m.,
évier n.m.,
four n.m.,
intérieur n.m.,
kit n.m.,
lavabo n.m.,
lave-linge n.m.,
lave-vaisselle n.m.,
lessive n.f.,
linge n.m.,
lit n.m.,
ménage n.m.,
meuble n.m.,
miroir n.m.,
poubelle n-f.

In [64]:
def get_response(prompt: str):

    client = OpenAI()

    response = client.completions.create(
        model="text-davinci-003",
        prompt=prompt, 
        max_tokens=1024,
        temperature=0
    )

    return response

In [65]:
response = get_response(prompt)

In [66]:
response_text = response.choices[0].text

In [67]:
print(response_text)

 {
        Le Vocabulaire: El Vocabulario,
        L'ampoule: La Bombilla,
        L'appareil: El aparato,
        L'aspirateur: La aspiradora,
        La baignoire: La bañera,
        Le bouton: El botón,
        Le bricolage: La bricolaje,
        Le bricoleur: El bricoleur,
        Le couvert: La cubierta,
        L'égalité: La igualdad,
        L'entretien: El mantenimiento,
        L'évier: El fregadero,
        Le four: El horno,
        L'intérieur: El interior,
        Le kit: El kit,
        Le lavabo: El lavabo,
        Le lave-linge: La lavadora,
        Le lave-vaisselle: El lavavajillas,
        La lessive: El detergente,
        Le linge: La ropa,
        Le lit: La cama,
        Le ménage: El hogar,
        Le meuble: El mueble,
        Le miroir: El espejo,
        La poubelle: La basura
    }


In [81]:
def output_to_csv(text: str):

    text = re.sub(r'[{},]','', text)

    output = text.split("\n")

    src = []
    dst = []

    for item in output: 
        if item.isspace() or not item:
            continue

        item = re.sub(r'\s+', ' ', item).strip()
        words = item.split(': ')

        src.append(words[0])
        dst.append(words[1])

    df = pd.DataFrame({
        'src': src,
        'dst': dst,
    })

    # Avoid the last '\n'
    return df.to_csv(index=False)

In [83]:
df = output_to_csv(response_text)